In [1]:
import tensorflow.compat.v1 as tf
!pip install transformers
tf.disable_eager_execution()
tf.test.gpu_device_name()

'/device:GPU:0'

In [0]:
# from google.colab import files
# files.upload()
# !pip install -q kaggle
# !mkdir -p ~/.kaggle
# !cp kaggle.json ~/.kaggle/
# !kaggle datasets download -d uciml/sms-spam-collection-dataset
# !unzip sms-spam-collection-dataset.zip

In [3]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from tqdm.notebook import tqdm
nltk.download('punkt')
nltk.download('stopwords')
tokenizer = RegexpTokenizer(r'\w+')
tqdm.pandas()

MAX_LEN = 128
def make_clean(s) :
  return s

df = pd.read_csv('spam.csv', encoding = "ISO-8859-1")
df['clean'] = df['v2'].progress_apply(make_clean)
df['label'] = (df['v1']=='ham').astype(int)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
from transformers import BertTokenizer
btokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

MAX_LEN = 128
input_ids = []
attention_masks = []

for sent in tqdm(df['clean']):
    encoded_dict = btokenizer.encode_plus(
                        sent,
                        add_special_tokens = True,
                        max_length = MAX_LEN,
                        pad_to_max_length = True,
                        return_attention_mask = True,
                        return_tensors = 'pt',
                   )
    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])

df['input_ids'] = input_ids
df['attention_masks'] = attention_masks

In [5]:
random_index = np.zeros(df.shape[0]).astype(bool)
random_index[np.random.choice(df.shape[0], int(0.2*df.shape[0]))] = True
train_df, test_df = df[~random_index], df[random_index]
train_df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4,clean,label,input_ids,attention_masks
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN,"Go until jurong point, crazy.. Available only ...",1,"[[tensor(101), tensor(2175), tensor(2127), ten...","[[tensor(1), tensor(1), tensor(1), tensor(1), ..."
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN,U dun say so early hor... U c already then say...,1,"[[tensor(101), tensor(1057), tensor(24654), te...","[[tensor(1), tensor(1), tensor(1), tensor(1), ..."
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN,"Nah I don't think he goes to usf, he lives aro...",1,"[[tensor(101), tensor(20976), tensor(1045), te...","[[tensor(1), tensor(1), tensor(1), tensor(1), ..."
5,spam,FreeMsg Hey there darling it's been 3 week's n...,NaN,NaN,NaN,FreeMsg Hey there darling it's been 3 week's n...,0,"[[tensor(101), tensor(2489), tensor(5244), ten...","[[tensor(1), tensor(1), tensor(1), tensor(1), ..."
6,ham,Even my brother is not like to speak with me. ...,NaN,NaN,NaN,Even my brother is not like to speak with me. ...,1,"[[tensor(101), tensor(2130), tensor(2026), ten...","[[tensor(1), tensor(1), tensor(1), tensor(1), ..."


In [0]:
import tensorflow_hub as hub
from tensorflow.compat.v1.keras import backend as K
# https://github.com/strongio/keras-bert/blob/master/keras-bert.ipynb
class BertLayer(tf.keras.layers.Layer):
    def __init__(
        self,
        n_fine_tune_layers=10,
        pooling="first",
        bert_path="https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1",
        **kwargs,
    ):
        self.n_fine_tune_layers = n_fine_tune_layers
        self.trainable = True
        self.output_size = 768
        self.pooling = pooling
        self.bert_path = bert_path
        if self.pooling not in ["first", "mean"]:
            raise NameError(
                f"Undefined pooling type (must be either first or mean, but is {self.pooling}"
            )

        super(BertLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.bert = hub.Module(
            self.bert_path, trainable=self.trainable, name=f"{self.name}_module"
        )

        # Remove unused layers
        trainable_vars = self.bert.variables
        if self.pooling == "first":
            trainable_vars = [var for var in trainable_vars if not "/cls/" in var.name]
            trainable_layers = ["pooler/dense"]

        elif self.pooling == "mean":
            trainable_vars = [
                var
                for var in trainable_vars
                if not "/cls/" in var.name and not "/pooler/" in var.name
            ]
            trainable_layers = []
        else:
            raise NameError(
                f"Undefined pooling type (must be either first or mean, but is {self.pooling}"
            )

        # Select how many layers to fine tune
        for i in range(self.n_fine_tune_layers):
            trainable_layers.append(f"encoder/layer_{str(11 - i)}")

        # Update trainable vars to contain only the specified layers
        trainable_vars = [
            var
            for var in trainable_vars
            if any([l in var.name for l in trainable_layers])
        ]

        # Add to trainable weights
        for var in trainable_vars:
            self._trainable_weights.append(var)

        for var in self.bert.variables:
            if var not in self._trainable_weights:
                self._non_trainable_weights.append(var)

        super(BertLayer, self).build(input_shape)

    def call(self, inputs):
        inputs = [K.cast(x, dtype="int32") for x in inputs]
        input_ids, input_mask, segment_ids = inputs
        bert_inputs = dict(
            input_ids=input_ids, input_mask=input_mask, segment_ids=segment_ids
        )
        if self.pooling == "first":
            pooled = self.bert(inputs=bert_inputs, signature="tokens", as_dict=True)[
                "pooled_output"
            ]
        elif self.pooling == "mean":
            result = self.bert(inputs=bert_inputs, signature="tokens", as_dict=True)[
                "sequence_output"
            ]

            mul_mask = lambda x, m: x * tf.expand_dims(m, axis=-1)
            masked_reduce_mean = lambda x, m: tf.reduce_sum(mul_mask(x, m), axis=1) / (
                    tf.reduce_sum(m, axis=1, keepdims=True) + 1e-10)
            input_mask = tf.cast(input_mask, tf.float32)
            pooled = masked_reduce_mean(result, input_mask)
        else:
            raise NameError(f"Undefined pooling type (must be either first or mean, but is {self.pooling}")

        return pooled

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_size)

In [0]:
class NonMasking(tf.keras.layers.Layer):   
    def __init__(self, **kwargs):   
        self.supports_masking = True  
        super(NonMasking, self).__init__(**kwargs)   
  
    def build(self, input_shape):   
        input_shape = input_shape   
  
    def compute_mask(self, input, input_mask=None):   
        # do not pass the mask to the next layers   
        return None   
  
    def call(self, x, mask=None):   
        return x   
  
    def get_output_shape_for(self, input_shape):   
        return input_shape  

In [8]:
from tensorflow.compat.v1.keras import layers
from tensorflow.compat.v1.keras.models import Model
from tensorflow.compat.v1.keras.optimizers import Adam

input_ids = layers.Input(shape=(MAX_LEN,), dtype=tf.int32, name="input_ids")
input_mask = layers.Input(shape=(MAX_LEN,), dtype=tf.int32, name="input_mask")
segment_ids = layers.Input(shape=(MAX_LEN,), dtype=tf.int32, name="segment_ids")
bert_inputs = [input_ids, input_mask, segment_ids]
bert_output = BertLayer(n_fine_tune_layers=10, pooling="first")(bert_inputs)
densembd = layers.Dense(768, activation='relu')(bert_output)
nmsk = NonMasking()(densembd)
rshp = layers.Reshape((-1,768))(nmsk)
convolution = layers.Convolution1D(50, 3, padding='same', activation='relu')(rshp)
convolution = layers.GlobalMaxPooling1D()(convolution)
dense = layers.Dense(50, activation='relu')(convolution)
pred = layers.Dense(1, activation='sigmoid')(dense)
model = Model(inputs=bert_inputs, outputs=pred)
model.compile(loss='binary_crossentropy', optimizer=Adam(lr=2e-4), metrics=['accuracy'])
model.summary()

INFO:absl:Using /tmp/tfhub_modules to cache modules.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 128)]        0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 128)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 128)]        0                                            
__________________________________________________________________________________________________
bert_layer (BertLayer)          (None, 768)          110104890   input_ids[0][0]                  
                                                                 input_mask[0][0]             

In [0]:
from tensorflow.compat.v1 import keras

session = keras.backend.get_session()
init = tf.global_variables_initializer()
session.run(init)

In [0]:
import numpy as np
import sklearn.metrics as sklm
from tensorflow.compat.v1 import keras

class Metrics(keras.callbacks.Callback):
    def __init__(self, val_data, batch_size = 32) :
        super().__init__()
        self.validation_data = val_data
        self.batch_size = batch_size

    def on_train_begin(self, logs={}):
        self.loss = []
        self.precision = []
        self.recall = []
        self.f1s = []
        self.accuracy = []
        self.auc = []

    def on_epoch_end(self, epoch, logs={}):
        score = np.asarray(self.model.predict(self.validation_data[0]))
        predict = np.squeeze(score.round()).reshape(-1)
        targ = self.validation_data[1]
        self.loss.append(logs['val_loss'])
        self.auc.append(sklm.roc_auc_score(targ, score))
        self.precision.append(sklm.precision_score(targ, predict))
        self.recall.append(sklm.recall_score(targ, predict))
        self.f1s.append(sklm.f1_score(targ, predict))
        self.accuracy.append(sklm.accuracy_score(targ, predict))

        pd.DataFrame({
            'loss': self.loss,
            'precision': self.precision,
            'recall': self.recall,
            'f1s': self.f1s,
            'accuracy': self.accuracy,
            'auc': self.auc
        }).to_csv('recors.csv')

        return

In [0]:
import numpy as np

# Create datasets (Only take up to MAX_LEN words)
train_ids = train_df['input_ids'].tolist()
train_ids = [t.tolist()[0:MAX_LEN] for t in train_ids]
train_ids = np.array(train_ids, dtype=int)[:, np.newaxis].reshape(train_df.shape[0],-1)
train_masks = train_df['attention_masks'].tolist()
train_masks = [t.tolist()[0:MAX_LEN] for t in train_masks]
train_masks = np.array(train_masks, dtype=int)[:, np.newaxis].reshape(train_df.shape[0],-1)
train_label = train_df['label'].tolist()

test_ids = test_df['input_ids'].tolist()
test_ids = [t.tolist()[0:MAX_LEN] for t in test_ids]
test_ids = np.array(test_ids, dtype=int)[:, np.newaxis].reshape(test_df.shape[0],-1)
test_masks = test_df['attention_masks'].tolist()
test_masks = [t.tolist()[0:MAX_LEN] for t in test_masks]
test_masks = np.array(test_masks, dtype=int)[:, np.newaxis].reshape(test_df.shape[0],-1)
test_label = test_df['label'].tolist()

In [0]:
# tf.compat.v1.experimental.output_all_intermediates(True)

In [13]:
  # def variable_created_in_scope(self, v):
  #   if not hasattr(v, '_distribute_strategy'):
  #     v._distribute_strategy = None
  #   return v._distribute_strategy is None  # pylint: disable=protected-access
#==============================================================================================
metrics = Metrics(([test_ids, test_masks, np.zeros(test_ids.shape)], test_label))

model.fit([train_ids, train_masks, np.zeros(train_ids.shape)], 
          train_label,
          validation_data=([test_ids, test_masks, np.zeros(test_ids.shape)], test_label),
          epochs=50,
          callbacks=[metrics],
          batch_size=32)

Train on 4568 samples, validate on 1004 samples
Epoch 1/50
4568/4568 [==============================] - 111s 24ms/sample - loss: 0.1581 - accuracy: 0.9532 - val_loss: 0.0939 - val_accuracy: 0.9801
Epoch 2/50
4568/4568 [==============================] - 113s 25ms/sample - loss: 0.1145 - accuracy: 0.9713 - val_loss: 0.1950 - val_accuracy: 0.9612
Epoch 3/50
4568/4568 [==============================] - 113s 25ms/sample - loss: 0.4455 - accuracy: 0.8430 - val_loss: 0.4379 - val_accuracy: 0.8416
Epoch 4/50
4568/4568 [==============================] - 113s 25ms/sample - loss: 0.2889 - accuracy: 0.9081 - val_loss: 0.4372 - val_accuracy: 0.8416
Epoch 5/50
4568/4568 [==============================] - 113s 25ms/sample - loss: 0.3863 - accuracy: 0.8713 - val_loss: 0.4568 - val_accuracy: 0.8416
Epoch 6/50
4568/4568 [==============================] - 113s 25ms/sample - loss: 0.3863 - accuracy: 0.8713 - val_loss: 0.4410 - val_accuracy: 0.8416
Epoch 7/50
4568/4568 [==============================] - 11

KeyboardInterrupt: ignored